In [41]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import StratifiedKFold

In [42]:
IMAGE_SIZE = [224, 224]
train_path="../pokemon_dataset/"

In [43]:
train = []
for file in os.listdir(os.path.join(train_path, 'train/pikachu')):
    train.append(['train/pikachu/{}'.format(file), 'pikachu'])

for file in os.listdir(os.path.join(train_path, 'train/raichu')):
    train.append(['train/raichu/{}'.format(file), 'raichu'])
    
train = pd.DataFrame(train, columns=['file', 'class_id'])

train = train.sample(frac=1).reset_index(drop=True)
# control data size
#train = train.sample(80).reset_index(drop=True)
#print(train)

In [44]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras import Sequential

In [45]:
model = Sequential()
model.add(Conv2D(32, (5,5),padding='valid',activation='relu',input_shape=(224,224,4)))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [46]:
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam
opti = Adam(learning_rate=0.001)
model.compile(
  optimizer=opti,
  loss=CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.Precision()])

In [47]:
from keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip=True,
                             rotation_range=30)

In [ ]:
from sklearn.model_selection import StratifiedKFold

strtfdKFold = StratifiedKFold(n_splits=10)
t = train.class_id
kfold = strtfdKFold.split(train, t)
scores = []
tot_score = 0

for k, (train_idx, test_idx) in enumerate(kfold):
    # prepare an iterators for each dataset
    train_data = train.loc[train_idx]
    test_data = train.loc[test_idx]
    train_it = datagen.flow_from_dataframe(dataframe = train_data,
                                       directory=train_path, 
                                       class_mode='categorical',
                                       x_col="file",
                                       y_col="class_id",
                                       target_size=(224, 224),
                                       color_mode='rgba',
                                       batch_size=2,
                                       seed=7)
    test_it = datagen.flow_from_dataframe(dataframe = test_data,  
                                       directory=train_path,
                                       class_mode='categorical',
                                       x_col="file",
                                       y_col="class_id",
                                       target_size=(224, 224),
                                       color_mode='rgba',
                                       batch_size=2,
                                       seed=7)

    model.fit(train_it,
            validation_data = test_it,
            epochs = 1)
    scores = model.evaluate(test_it)
    tot_score += scores[1]*100
    print("Accuarcy %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

print('\n\nCross-Validation accuracy: %.3f' %(tot_score/10))